In [1]:
import cv2
import numpy as np
import os

## Procedure
- Read and show video stream , capture images
- Detect Fces and show bounding box
- Flatten the Largest face image and save in numpy array
- Repeat the above for multiple people to generate training data

In [2]:
#calculate the Euclidean distance between two vectors
def distance(v1, v2):
    # Eucledian 
    return np.sqrt(((v1-v2)**2).sum())

def knn(train, test, k=5):
    dist = []
    
    for i in range(train.shape[0]):
        # Get the vector and label
        ix = train[i, :-1]
        iy = train[i, -1]
        # Compute the distance from test point
        d = distance(test, ix)
        dist.append([d, iy])
    # Sort based on distance and get top k
    dk = sorted(dist, key=lambda x: x[0])[:k]
    # Retrieve only the labels
    labels = np.array(dk)[:, -1]
    
    # Get frequencies of each label
    output = np.unique(labels, return_counts=True)
    # Find max frequency and corresponding label
    index = np.argmax(output[1])
    return output[0][index]


In [ ]:
#init camera
cap = cv2.VideoCapture(0)

#face detection
face_cascade_classifier = cv2.CascadeClassifier("haarcascade_frontalface_alt.xml")
skip = 0

#face_data is training data
face_data = []
labels=[]
#load datset
dataset_path = "./data/"

class_id = 0 
names = {} #mapping between id and name

#Data Preparation
for fx in os.listdir(dataset_path):
    if fx.endswith('.npy'):
        names[class_id] = fx[:-4]
        print('loaded'+fx)
        data_item = np.load(dataset_path+fx)
        face_data.append(data_item)
        #Create Labels for the class
        target = class_id*np.ones((data_item.shape[0],))
        class_id+=1
        labels.append(target)
face_dataset = np.concatenate(face_data,axis=0)
face_label = np.concatenate(labels,axis=0).reshape((-1,1))
trainset = np.concatenate((face_dataset,face_label),axis=1)
print(trainset.shape)
#Testing
face_cascade_classifier = cv2.CascadeClassifier("haarcascade_frontalface_alt.xml")
while True:
    ret,frame = cap.read()
    if ret == False:
        continue
    faces = face_cascade_classifier.detectMultiScale(frame,1.3,5)
    
    for face in faces:
        x,y,w,h = face
        #region of interest
        offset = 10
        frame_section = frame[y-offset:y+h+offset,x-offset:x+w+offset]
        frame_section = cv2.resize(frame_section,(100,100))
        #predicted label
      
        out = knn(trainset,frame_section.reshape(1,-1))
        pred_name = names[int(out)]
        cv2.putText(frame,pred_name,(x,y-10),cv2.FONT_HERSHEY_SIMPLEX,1,(255,0,0),2,cv2.LINE_AA)
        cv2.rectangle(frame,(x,y),(x+w,y+h),(0,255,255),2)
    cv2.imshow("frame",frame)
    key_pressed = cv2.waitKey(1)&0XFF
    if key_pressed == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()       

loadedaditya.npy
loadedarmaan.npy
loadedRitik.npy
(64, 30001)
